In [1]:
import matplotlib.pyplot as plt
import teetool as tt
import pandas as pd
import numpy as np

import mayavi.mlab as mlab

/home/will/anaconda3/envs/tt/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# create a world and add existing data

# new world -- nres defines default resolution of tube/logp grid generation
world = tt.World(name="crs31", ndim=3)

filename = ("1_standard_launch")

# extract data from pickle
cluster_data = pd.read_pickle("temp/{0}.pickle".format(filename));

# add data to the world
world.addCluster(cluster_data, filename)

In [3]:
world.buildModel(settings={"model_type":"resampling", "ngaus":100})
world.overview()

*** overview [crs31] ***
0 [1_standard_launch] [*]


In [4]:
# obtain list of points of ellipsoids
icluster = 0

# extract
this_cluster = world._clusters[icluster]

# obtain cells
cc = this_cluster["model"]._cc
cA = this_cluster["model"]._cA

# obtain points ellipsoid
#E_list = this_cluster["model"]._get_point_cloud(sdwidth=1, nsamples=50)
Ex_list = np.linspace(0, 1, len(cc))



In [5]:
# vary numbers

for Ei in range(25):
    
    # visualise time-series
    ndim = 3
    ntraj=10
    
    c = cc[Ei]
    A = cA[Ei]

    # Ei = 50
    E = this_cluster["model"]._getEllipse(c, A, sdwidth=1, npoints=50)
    E = tt.helpers.unique_rows(E)
    Ex = Ex_list[Ei]*np.ones_like(E[:,0])
    
    #print(E.shape)

    # subplot
    f, axarr = plt.subplots(ndim, sharex=True, facecolor="white", dpi=600, figsize=(7, 5))

    # extract data
    clusters = world.getCluster([icluster])
    this_cluster = clusters[0]
    
    Y = this_cluster["model"].getMean()
    
    for d in range(ndim):
        
        x = np.linspace(0, 1, np.size(Y, 0))
        
        axarr[d].plot(x, Y[:,d],color='b')
        
        """
        for itraj, (x, Y) in enumerate(this_cluster["data"]):

            x_norm = (x - x.min()) / (x.max() - x.min())
            axarr[d].plot(x_norm, Y[:,d],color='b')

            if itraj > ntraj:
                break
        """
        
        # ellipse values
        axarr[d].plot(Ex, E[:,d],'.r')

    axarr[0].set_ylabel('Easting [m]')
    axarr[1].set_ylabel('Northing [m]')
    axarr[2].set_ylabel('Altitude [m]')

    axarr[0].set_ylim([-1000, 1000])
    axarr[1].set_ylim([0, 3000])
    axarr[2].set_ylim([0, 3000])

    axarr[0].set_yticks([-1000, 0, 1000])
    axarr[1].set_yticks([0, 1500, 3000])
    axarr[2].set_yticks([0, 1500, 3000])

    axarr[2].set_xlabel('normalised time [-]')

    plt.savefig("output/3d_crs31_ellipsoid_{0}_ts.png".format(Ei))

    plt.close()
    
    # visualise Ellipsoid
    
    visual = tt.visual_3d.Visual_3d(world, size=(1200, 900))

    # visualise mean
    visual.plotMean([icluster], colour=(0.0, 0.0, 0.8))

    # visualise ellipsoid
    mlab.points3d(E[:,0], E[:,1], E[:,2], scale_factor=5, color=(0.8,0.0,0.0))

    # 1000m = 1km = 1 km^3 is one grid block
    visual.plotGrid(resolution=500,
                    outline=[-1000, 1000, 0, 3000, 0, 3000])

    # labels and ticks
    visual.setLabels("Easting [m]", "Northing [m]", "Altitude [m]")
    visual.setAxesFormat("%.0f")  # no ticks

    # change view
    visual.setView(azimuth=10, elevation=87, focalpoint=[0, 1400, 1300], distance=9200)

    visual.save(add="ellipsoid_{0}".format(Ei))

    # visual.show()
    visual.close()

In [6]:
# visualise time-series
ndim = 3
ntraj=10

Ei_list = range(99)

#print(E.shape)

# subplot
f, axarr = plt.subplots(ndim, sharex=True, facecolor="white", dpi=600, figsize=(7, 5))

# extract data
clusters = world.getCluster([icluster])
this_cluster = clusters[0]

Y = this_cluster["model"].getMean()

x = np.linspace(0, 1, np.size(Y, 0))

for d in range(ndim):
    
    axarr[d].plot(x, Y[:,d],color='b')
    
    """
    for itraj, (x, Y) in enumerate(this_cluster["data"]):

        x_norm = (x - x.min()) / (x.max() - x.min())
        axarr[d].plot(x_norm, Y[:,d],color='b')

        if itraj > ntraj:
            break
    """

for Ei in Ei_list:
    
    c = cc[Ei]
    A = cA[Ei]
    
    # Ei = 50
    E = this_cluster["model"]._getEllipse(c, A, sdwidth=1, npoints=50)
    E = tt.helpers.unique_rows(E)
    Ex = Ex_list[Ei]*np.ones_like(E[:,0])

    for d in range(ndim):
        # ellipse values
        axarr[d].plot(Ex, E[:,d],'.r', markersize=1)

axarr[0].set_ylabel('Easting [m]')
axarr[1].set_ylabel('Northing [m]')
axarr[2].set_ylabel('Altitude [m]')

axarr[0].set_ylim([-1000, 1000])
axarr[1].set_ylim([0, 3000])
axarr[2].set_ylim([0, 3000])

axarr[0].set_yticks([-1000, 0, 1000])
axarr[1].set_yticks([0, 1500, 3000])
axarr[2].set_yticks([0, 1500, 3000])

axarr[2].set_xlabel('normalised time [-]')

plt.savefig("output/3d_crs31_ellipsoid_ALL_ts.png")

plt.close()

# visualise Ellipsoid

visual = tt.visual_3d.Visual_3d(world, size=(1200, 900))

# visualise mean
visual.plotMean([icluster], colour=(0.0, 0.0, 0.8))

# visualise ellipsoid

for Ei in Ei_list:
    
    c = cc[Ei]
    A = cA[Ei]
    
    # Ei = 50
    E = this_cluster["model"]._getEllipse(c, A, sdwidth=1, npoints=50)
    E = tt.helpers.unique_rows(E)

    mlab.points3d(E[:,0], E[:,1], E[:,2], scale_factor=5, color=(0.8,0.0,0.0))

# 1000m = 1km = 1 km^3 is one grid block
visual.plotGrid(resolution=500,
                outline=[-1000, 1000, 0, 3000, 0, 3000])

# labels and ticks
visual.setLabels("Easting [m]", "Northing [m]", "Altitude [m]")
visual.setAxesFormat("%.0f")  # no ticks

# change view
visual.setView(azimuth=10, elevation=87, focalpoint=[0, 1400, 1300], distance=9200)

visual.save(add="ellipsoid_ALL")

# visual.show()
visual.close()